<a href="https://colab.research.google.com/github/yuichi0625/cellar/blob/main/check_country_language_pair_on_transformers/bert_uncased_multilingual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 2.0MB 4.2MB/s 
     |████████████████████████████████| 890kB 32.1MB/s 
     |████████████████████████████████| 3.2MB 43.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=428b0780f272542bd4ec0d7beef12e33b29fa6f90d2b27a19864f8c30e9cb678
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
!wget https://raw.githubusercontent.com/yuichi0625/cellar/main/check_country_language_pair_on_transformers/countries.csv

--2021-03-23 16:34:18--  https://raw.githubusercontent.com/yuichi0625/cellar/main/check_country_language_pair_on_transformers/countries.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7068 (6.9K) [text/plain]
Saving to: ‘countries.csv’

countries.csv       100%[===================>]   6.90K  --.-KB/s    in 0s      

2021-03-23 16:34:19 (68.5 MB/s) - ‘countries.csv’ saved [7068/7068]



In [3]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM
pd.set_option('display.max_rows', 500)

tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-uncased')
model = AutoModelForMaskedLM.from_pretrained('bert-base-multilingual-uncased')

sequence = 'I live in {} and I speak {}.'

rows = []
for name in pd.read_csv('countries.csv')['en'].sort_values():
  input = tokenizer.encode(sequence.format(name, tokenizer.mask_token), return_tensors='pt')
  mask_token_index = torch.where(input == tokenizer.mask_token_id)[1]

  token_logits = model(input).logits[0]
  mask_token_logits = token_logits[mask_token_index, :][0]
  mask_token_probs = torch.softmax(mask_token_logits, dim=0)

  candidate_indices = torch.topk(mask_token_logits, k=3, dim=0).indices.tolist()
  result = [f'{tokenizer.decode([ci])} ({mask_token_probs[ci] * 100:.0f}%)' for ci in candidate_indices]

  rows.append([name, *result])

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
pd.DataFrame(rows, columns=['country', 'language(top1)', 'language(top2)', 'language(top3)'])

,country,language(top1),language(top2),language(top3)
0,Afghanistan,english (31%),russian (7%),french (7%)
1,Albania,english (32%),albanian (10%),italian (8%)
2,Algeria,french (47%),english (15%),arabic (13%)
3,Andorra,spanish (25%),english (24%),french (18%)
4,Angola,portuguese (21%),english (19%),afrikaans (11%)
5,Antigua and Barbuda,english (48%),spanish (12%),french (8%)
6,Argentina,english (32%),spanish (29%),french (7%)
7,Armenia,armenian (57%),english (9%),russian (9%)
8,Australia,english (54%),french (4%),spanish (3%)
9,Austria,german (22%),english (21%),french (8%)


In [5]:
sequence = 'Ich wohne in {}, und ich spreche {}.'

rows = []
for name in pd.read_csv('countries.csv')['de'].sort_values():
  input = tokenizer.encode(sequence.format(name, tokenizer.mask_token), return_tensors='pt')
  mask_token_index = torch.where(input == tokenizer.mask_token_id)[1]

  token_logits = model(input).logits[0]
  mask_token_logits = token_logits[mask_token_index, :][0]
  mask_token_probs = torch.softmax(mask_token_logits, dim=0)

  candidate_indices = torch.topk(mask_token_logits, k=3, dim=0).indices.tolist()
  result = [f'{tokenizer.decode([ci])} ({mask_token_probs[ci] * 100:.0f}%)' for ci in candidate_indices]

  rows.append([name, *result])

In [6]:
pd.DataFrame(rows, columns=['Land', 'Sprache(top1)', 'Sprache(top2)', 'Sprache(top3)'])

,Land,Sprache(top1),Sprache(top2),Sprache(top3)
0,Afghanistan,dich (14%),englisch (8%),deutsch (5%)
1,Albanien,dich (11%),deutsch (10%),englisch (9%)
2,Algerien,dich (18%),franzosisch (12%),englisch (5%)
3,Andorra,franzosisch (14%),dich (10%),englisch (8%)
4,Angola,englisch (10%),deutsch (7%),dich (7%)
5,Antigua und Barbuda,englisch (24%),franzosisch (7%),deutsch (5%)
6,Argentinien,englisch (10%),dich (8%),nicht (6%)
7,Armenien,dich (15%),russisch (8%),englisch (5%)
8,Aserbaidschan,dich (19%),russisch (8%),englisch (4%)
9,Australien,englisch (14%),dich (9%),deutsch (9%)
